In [ ]:
# import statements
import requests
import re
import json
import time
from bs4 import BeautifulSoup as BS

In [ ]:
def scrape(url):
    # need header to scrape from wikipedia
    headers = {
        'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
    }
    page = requests.get(url, headers=headers)
    
    time.sleep(3)
    
    return BS(page.text, "html")

In [ ]:
def write_json(dict):
    with open(f'data/rider_teams.json', 'r+') as file:
        # load the entire json file into memory
        file_json = json.load(file)

        # update the json with the python dictionary
        file_json['rider'].append(dict)

        # put cursor at beginning of file and overwrite it with the dictionary in memory
        file.seek(0)
        json.dump(file_json, file, indent=4)

In [ ]:
def get_team_links():
    soup = scrape('https://www.procyclingstats.com/race/tour-de-france')

    team_links = [
        team.select('a')['href']

        for team in soup.find('ul', class_='topnav').select('li')[2].find('ul').select('li')
    ]

    return team_links

In [ ]:
def scrape_team(team_url):
    # scrape the team page first
    soup = scrape(team_url)

    # extract all rider names and links
    # why class_ and not class? TODO
    riders = [
        {'name': rider.find('div', class_='name').get_text(strip=True), 'link': soup.find('a')['href']}

        for rider in soup.find('ul', class_='name').select('li')
    ]

    for rider, i in enumerate(riders, start=0):
        soup = scrape(rider['link'])

        rider_teams = {
            team.select('div', class_='name').get_text(strip=True)
            
            for team in soup.find('ul', class_='rdr-teams2').select('li')
        }

        riders[i].update({'teams': rider_teams})

    write_json(riders)

In [ ]:
links = get_team_links()

[scrape_team(link) for link in links]